# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [141]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dataset.zip
replace Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [142]:
!pip install chromadb

ERROR: Operation cancelled by user


In [143]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached Chroma-0.2.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


In [144]:
!pip install sentence-transformers

In [145]:
%pip install -qU langchain-experimental

In [160]:
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import markdown
from langchain_groq import ChatGroq
from langchain.chains import LLMChain

# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [147]:
df = pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [148]:
len(df)

104

## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [161]:
#Create Markdown Content from the Dataset
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [162]:
for i in range(0,103):

     title = df["المخالفة"].iloc[i]
     content = df["الغرامة"].iloc[i]

     markdown_content = f"# {title}\n\n"
     markdown_content += f"{content}\n\n"

     with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

In [163]:
markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)

## Step 4: Chunk the Markdown Data

In [152]:
!pip install tiktoken

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [166]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,
                                               chunk_overlap=20,
                                               length_function=len,
                                               is_separator_regex=False,
                                               )
text = text_splitter.create_documents(markdown_texts)
print(text)

[Document(metadata={}, page_content='<h1>طمس لوحات المركبة.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>'), Document(metadata={}, page_content='<h1>قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.</h1>\n<p>الغرامة المالية 100 - 150 ريال</p>'), Document(metadata={}, page_content='<h1>عدم ربط حزام الأمان.</h1>\n<p>الغرامة المالية 150 - 300 ريال</p>'), Document(metadata={}, page_content='<h1>استعمال المركبة لغير الغرض الذي رخصت من أجله.</h1>\n<p>الغرامة المالية 500 - 900 ريال</p>'), Document(metadata={}, page_content='<h1>القيادة على أكتاف الطريق وعلى الأرصفة أو المسارات التي يمنع القيادة فيها.</h1>'), Document(metadata={}, page_content='<p>الغرامة المالية 1000 - 2000 ريال</p>'), Document(metadata={}, page_content='<h1>عدم قيام السائق الذي يرغب في تغيير مساره بإعطاء الأفضلية لسائق المركبة التي تسير في اتجاه مستقيم'), Document(metadata={}, page_content='في اتجاه مستقيم في حال سير المركبتين متحاذيتين بشكل متواز.</h1>'), Document(metadata={}, page_content='<p>الغرامة المالية 500 - 900 ر

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [154]:
!pip install -U langchain-community

In [170]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="asafaya/bert-base-arabic")
db = Chroma(text,embedding_function, persist_directory="./chroma_db")
# "aubmindlab/bert-large-arabertv02" # embedding

ValueError: Expected collection name that (1) contains 3-63 characters, (2) starts and ends with an alphanumeric character, (3) otherwise contains only alphanumeric characters, underscores or hyphens (-), (4) contains no two consecutive periods (..) and (5) is not a valid IPv4 address, got [Document(metadata={}, page_content='<h1>طمس لوحات المركبة.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>'), Document(metadata={}, page_content='<h1>قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.</h1>\n<p>الغرامة المالية 100 - 150 ريال</p>'), Document(metadata={}, page_content='<h1>عدم ربط حزام الأمان.</h1>\n<p>الغرامة المالية 150 - 300 ريال</p>'), Document(metadata={}, page_content='<h1>استعمال المركبة لغير الغرض الذي رخصت من أجله.</h1>\n<p>الغرامة المالية 500 - 900 ريال</p>'), Document(metadata={}, page_content='<h1>القيادة على أكتاف الطريق وعلى الأرصفة أو المسارات التي يمنع القيادة فيها.</h1>'), Document(metadata={}, page_content='<p>الغرامة المالية 1000 - 2000 ريال</p>'), Document(metadata=

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [ ]:
PROMPT_TEMPLATE="""
Answer the question based only on the following context:
Context: {context_text}
Question: {query}
Your answer:
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [ ]:
!pip install langchain_groq

In [ ]:
groq_api_key = "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"

In [159]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.8,
    max_retries=2,
    api_key=groq_api_key
)

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [ ]:
from langchain.chains import LLMChain
MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [ ]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([text.page_content for text, _score in similarity_search_results])

    rag_response = MODEL.invoke({"context": context_text, "question": query})

    return rag_response

In [ ]:
response = query_rag("ماهي الغرامة على القيادة بدون رخصة؟")
response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.